## Soluciones lab07-challenge

Entorno

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as f
from pyspark.sql.window import Window
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

# Crear la sesión de Spark
spark = SparkSession.builder.appName("lab07").getOrCreate()

# Define el schema
schema = StructType([
    StructField("movieId", IntegerType(), True),
    StructField("title", StringType(), True),
    StructField("genres", StringType(), True),
    StructField("views", IntegerType(), True),
    StructField("quality", StringType(), True)
])

# Leer archivo CSV
movies = spark.read.option("header", True).schema(schema).csv("../../data/movies.csv")

Apartado 1

In [ ]:
df_exploded = movies.withColumn("genre", f.explode(f.split(f.col("genres"), "\|")))
window_spec = Window.partitionBy("genre").orderBy(f.col("views").desc())
df_top3 = df_exploded.withColumn("rank", f.row_number().over(window_spec)) \
                     .filter(f.col("rank") <= 3) \
                     .select("movieId", "title", "genre", "views")
df_top3.show()

In [ ]:
df_top3.write.partitionBy("genre").parquet("lab_data/partition_by_genre.parquet")

Apartado 2

In [ ]:
adventure_data = spark.read.parquet("./lab_data/partition_by_genre.parquet/genre=Adventure")
adventure_data.show()

In [38]:
spark.stop()